In [1]:
import sys
sys.path.append('/home/jovyan/workspace')

In [2]:
%%sh
rm -rf '../data/test'
mkdir '../data/test'
chmod -R 777 '../data/test'

In [3]:
import os
outputPath = '../data/test'

In [4]:
import planetengine
system = planetengine.systems.arrhenius.build(res = 16)
initials = {'temperatureField': planetengine.initials.sinusoidal.IC(freq = 1.)}
planetengine.initials.apply(
    initials,
    system,
    )
system.solve()

In [10]:
from planetengine.utilities import check_reqs
import underworld as uw

class Observer(_Frame):

    _required_attributes = {
        'system', # system
        'initials', # dict
        'outputPath', # str
        'scripts', # list of str
        'inputs', # list of dict
        }

    def __init__(self):

        check_reqs(self)

        scripts = {}
        for index, script in enumerate(self.scripts):
            scriptname = 'observerscript_' + str(index)
            scripts[scriptname] = script
        self.scripts = scripts

        inputs = {}
        for index, inputDict in enumerate(inputs):
            inputName = 'observerinputs_' + str(index)
            inputs[inputName] = inputDict
        self.inputs = inputs

        # Making stamps and stuff

        _model_inputs, _model_stamps, _model_scripts = \
            _scripts_and_stamps(system, initials)

        inHashID = 'pemod_' + _model_stamps['allstamp'][1]

        stamps = {}
        if uw.mpi.rank == 0:
            stamps = {
                'inputs': hashstamp(self.inputs),
                'scripts': hashstamp(
                    [open(script) for scriptname, script \
                        in sorted(scripts.items())]
                    )
                }
            stamps['allstamp'] = hashstamp(stamps)
            for stampKey, stampVal in stamps.items():
                stamps[stampKey] = [stampVal, wordhashFn(stampVal)]
        stamps = uw.mpi.comm.bcast(stamps, root = 0)

        super().__init__()

In [ ]:
class MyObserver(Observer):

    def __init__(
        self,
        _system,
        _initials,
        _outputPath,
        )
    

In [8]:
from planetengine.utilities import Grouper
from planetengine.utilities import hashstamp
from planetengine.wordhash import wordhash as wordhashFn
from planetengine.checkpoint import Checkpointer
from planetengine.frame import _scripts_and_stamps
from planetengine.frame import _Frame
import underworld as uw
#
def make_observer_stamps_inputs(system, initials, options):
    stamps = {}
    _model_inputs, _model_stamps, _model_scripts = \
        _scripts_and_stamps(system, initials)
    if uw.mpi.rank == 0:
        stamps.update(_model_stamps)
        stamps = {
            'options': hashstamp(options),
            'scripts': hashstamp(
                [open(script) for scriptname, script \
                    in sorted(scripts.items())]
                )
            }
        stamps['allstamp'] = hashstamp(stamps)
        for stampKey, stampVal in stamps.items():
            stamps[stampKey] = [stampVal, wordhashFn(stampVal)]
    stamps = uw.mpi.comm.bcast(stamps, root = 0)
    return stamps

class Observer(_Frame):

    figs = []
    collectors = []

    def __init__(
            self,
            system,
            initials,
            obsInps = {},
            subObservers = [],
            outputPath = '',
            _autoarchive = True,
            _parentFrame = None,
            _is_child = False,
            _autobackup = True,
            ):

        scripts = {}
        for index, subObserver in enumerate(subObservers):
            scriptname = 'observerscript_' + str(index)
            scripts[scriptname] = script

        # Making stamps and stuff

        _model_inputs, _model_stamps, _model_scripts = \
            _scripts_and_stamps(system, initials)

        self.inHashID = 'pemod_' + _model_stamps['allstamp'][1]

        stamps = {}
        if uw.mpi.rank == 0:
            stamps = {
                'obsInps': hashstamp(obsInps),
                'scripts': hashstamp(
                    [open(script) for scriptname, script \
                        in sorted(scripts.items())]
                    )
                }
            stamps['allstamp'] = hashstamp(stamps)
            for stampKey, stampVal in stamps.items():
                stamps[stampKey] = [stampVal, wordhashFn(stampVal)]
        stamps = uw.mpi.comm.bcast(stamps, root = 0)

        # Making the checkpointer:

        checkpointer = Checkpointer(
            stamps = stamps,
            step = system.step,
            modeltime = system.modeltime,
            figs = self.figs,
            dataCollectors = self.collectors,
            scripts = scripts,
            inputs = {'obsInps': obsInps}
            )

        self.checkpointer = checkpointer
        self.inputs = inputs
        self.scripts = scripts
        self.stamps = stamps
        self.system = system
        self.initials = initials
        self.step = system.step
        self.modeltime = system.modeltime
        self.instanceID = stamps['allstamp'][1]
        self.outputPath = outputPath

        super().__init__()

    def prompt(self):
        pass

    def checkpoint(self, path):
        self.checkpointer.checkpoint(path)

    def load_checkpoint(self, path):
        pass

In [9]:
observer = Observer(system, initials, ['./testfile.py'])

Making stamps...
Stamps made.


NameError: name 'inputs' is not defined

In [ ]:
path = os.path.join(outputPath, 'obstest')
observer.checkpoint(path)

In [ ]:
system.iterate()

In [ ]:
observer.checkpoint(path)